Adding PCA to fasten the algorithm

In [6]:
from sklearn.decomposition import PCA
import numpy as np
from derivative import *
from HW3_B2 import *
from HW3_B4 import several_newton
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve
from progress.bar import Bar
from alive_progress import alive_bar

In [7]:

def pca(vectors_A, vectors_B, n):
    # Combine the two arrays into a single array for PCA
    combined_vectors = np.concatenate((vectors_A, vectors_B))
    pca = PCA(n_components=n)
    # Fit and transform the combined vectors
    transformed_vectors = pca.fit_transform(combined_vectors)
    # Split the transformed data back into vectors_A and vectors_B
    new_vectors_A = transformed_vectors[:len(vectors_A)]
    new_vectors_B = transformed_vectors[len(vectors_A):]

    return new_vectors_A, new_vectors_B

Derivatives

B2

In [8]:
# Define the function to perform the optimization
def short_step_method(class_A, class_B, lambda_param, nu, epsilon):

    nA, n = class_A.vectors.shape
    nB = class_B.vectors.shape[0]

    C = c_coeff(n, nA, nB, lambda_param)

    tau = 1/4
    theta = 1/(16*np.sqrt(nu))

    h, c, s, t, v, mu = init(class_A, class_B)
    mu_final = epsilon * (1 - tau) / nu

    ite = 0
    with alive_bar(10000) as bar:
        while mu > mu_final and ite < 10000:
            mu *= (1 - theta)
            step = newton_step(h, c, s, t, v, class_A, class_B, mu, lambda_param, C)
            h, s, t, c, v = uptade(h, s, t, c, v, step)
            ite += 1
            bar()

    return h, c, s, t, v


Long-Step-Method (B4)

In [9]:
def long_step_method(class_A, class_B, lambda_param, nu, epsilon, theta):

    h, c, s, t, v, mu = init(class_A, class_B)

    nA, n = class_A.vectors.shape
    nB = class_B.vectors.shape[0]

    C = c_coeff(n, nA, nB, lambda_param)

    tau = .25

    mu_final = epsilon * (1 - tau) / nu

    ite = 0
    with alive_bar(10000) as bar:
        while mu > mu_final and ite < 10000:
            mu *= (1 - theta)
            h, s, t, c, v = several_newton(h, c, s, t, v, class_A, class_B, mu, lambda_param, C)
            ite += 1
            bar()


    return h, c, s, t, v



B3 - B4 (short step / long step)

In [10]:
from sklearn.metrics import accuracy_score

NIMAGE = 5 #will be multiplied by 9, can be changed
n_pca = 10

image_size = 28 # width and length
no_of_different_labels = 10
image_pixels = image_size * image_size
data_path = "./"
train_data = np.loadtxt(data_path + "mnist_train.csv", 
                        delimiter=",")
test_data = np.loadtxt(data_path + "mnist_test.csv", 
                       delimiter=",") 

fac = 0.99 / 255
train_imgs = np.asfarray(train_data[:, 1:]) * fac + 0.01
test_imgs = np.asfarray(test_data[:, 1:]) * fac + 0.01

train_labels = np.asfarray(train_data[:, :1])
test_labels = np.asfarray(test_data[:, :1])

vector_A = train_imgs[np.where(train_labels == 0)[0]]
vector_B = np.concatenate([train_imgs[np.where(train_labels == i)[0]][:(NIMAGE)] for i in range(1, 10)])
np.random.shuffle(vector_B)
vector_A = vector_A[:(NIMAGE*9)]

#PCA vectors to fasten the algorithm
#vector_A, vector_B = pca(vector_A, vector_B, n_pca)     #uncomment it to use PCA

# Initialize class instances
class_A = A(vector_A)
class_B = B(vector_B)

# Set the lambda parameter for the optimization problem
lambda_param = 1
nu = 2 * len(class_A.vectors) + 2 * len(class_B.vectors) + 2
epsilon = 1e-6

# h0, c0, s0, t0, v0  = init(class_A, class_B)
#h, c, s, t, v  = short_step_method(class_A, class_B, lambda_param, nu, epsilon)   #uncoment for the short-step method

# h0, c0, s0, t0, v0  = init(class_A, class_B)
h, c, s, t, v  = long_step_method(class_A, class_B, lambda_param, nu, epsilon, 0.99)

|██▋⚠︎                                    | (!) 66/1000 [7%] in 11.7s (5.64/s) 
|⚠︎                                       | (!) 5/10000 [0%] in 1:38.3 (0.05/s) 


C.1 Accuracy over training set

In [11]:
def global_accuracy(imgs, labels, classify, l):
    # Accuracy over the training set
    y_pred = np.array([classify(im, l) for im in imgs])
    labels = (labels > 0)*1.0
    print("Global Accuracy: ", accuracy_score(labels, y_pred))

def label_accuracy(imgs, labels, classify, l):
    #Accuracy for each label
    for i in range(10):
        y_pred = np.array([classify(im, l) for im in imgs[np.where(labels == i)[0]]])
        if i > 0: 
            true = np.ones(y_pred.shape[0])
        else:
            true = np.zeros(y_pred.shape[0])
        print("Accuracy for label "+ str(i) + ": " + str(accuracy_score(true, y_pred)))

In [14]:
Lambdas = [0.1, 1, 10, 100, 1000]
Thetas = [0.1, 0.5, 0.9, 0.99]
l = lambda x: np.dot(h, x) + c
def classify(x, l=l):
    return 0 if l(x) < -1 else 1

for lambda_ in Lambdas:
    h, c, s, t, v = short_step_method(class_A, class_B, lambda_, nu, epsilon)
    
    print("Lambda = " + str(lambda_))
    print("-------------Global Accuracy Over Training Set in function of Lambda------------------------")
    global_accuracy(train_imgs, classify, l)
    print("-------------Accuracy for each label Over Training Set in function of Lambda----------------")
    label_accuracy(train_imgs, train_labels, classify, l)

for theta in Thetas:
    h, c, s, t, v = long_step_method(class_A, class_B, lambda_param, nu, epsilon, theta)
    
    print("Theta = " + str(theta))
    print("-------------Global Accuracy Over Training Set in function of Theta------------------------")
    global_accuracy(train_imgs, classify, l)
    print("-------------Accuracy for each label Over Training Set in function of Theta----------------")
    label_accuracy(train_imgs, train_labels, classify, l)

|██▋⚠︎                                    | (!) 66/1000 [7%] in 12.8s (5.14/s) 
|▉⚠︎                                      | (!) 232/10000 [2%] in 41.7s (5.57/s) 


KeyboardInterrupt: 

C.2 Generalized Performance (accuracy over test set)

In [15]:
h, c, s, t, v  = long_step_method(class_A, class_B, lambda_param, nu, epsilon, 0.99) #To change for the best method, could be long-step or short step
l = lambda x: np.dot(h, x) + c

print("-------------Global Accuracy Over Test Set------------------------")
global_accuracy(test_imgs, test_labels, classify, l)
print("-------------Accuracy for each label Over Test Set----------------")
label_accuracy(test_imgs, test_labels, classify, l)

|██▍⚠︎                                    | (!) 59/1000 [6%] in 12.3s (4.80/s) 


KeyboardInterrupt: 